# 1.YOLOについて論文を読んだので学習内容まとめ

ちなみに、YOLOは新しいバージョンがどんどん出てきていて、２０１８年７月時点では「YOLO V3」が最新版なのかな。
本論文は2016年発表のもの。  
Pythonで使うためには「darknet」というフレームワークを利用する必要があり、tensorflow等で用意されている訳ではないのでsagemakerではまだ使えない。　　
もしかしたら、YOLOは商用利用不可のものかもしれない。  
商用利用するなら、[SSD(Single Shot MultiBox Detector)](https://qiita.com/yampy/items/37c607fdf77a919cda5d)というアルゴリズムがいいのかも？


##### 1-1.元論文
https://arxiv.org/pdf/1506.02640.pdf

# 2.YOLO
### 2-1.特徴
End-to-Endのネットワーク。
画像中の「物体」と「範囲」を識別するもの。
小さい物体の識別が苦手だが、処理速度が早く、ぼちぼちな精度を叩き出す。
（当時のstate-of-the-artな手法と比較できる精度、ということ）
処理が１つのネットワークで完結している。（BBの座標とクラス予測を一度に１つのネットワークで出力する）


### 2-2.処理内容の概要
inputは448*448*3の画像。outputは7*7*30。outputの形は「S × S × (B ∗ 5 + C) tensor」となる。
（元論文では、画像を7*7のgridに分割、各セルのバウンディングボックスを2個、クラスは２０分類としていたため。）
２４層の畳み込み層と、最後に２層の全結合層からなる。

S:画像を何個のgridに分割するのか
B:各gridにて、何個の物体まで処理させるか。
C:クラス分類のクラス数


なお、元論文でも「Fast- YOLO」というより処理が早いYOLOでも検証をしたみたいだが、こちらは畳み込み層が9層になり、フィルターも少なくした以外はYOLOと同じ。


### 2-3.処理内容の詳細
まず最初の20層の畳み込み層で事前学習をした。
この時点で、「ImageNet1000-class competition dataset」の「top-5 accuracy」で88%の結果だった。
この事前学習の後に、畳み込み層と全結合層を追加する形で本モデルを生成した。（初期パラメータは、乱数で適当に設定）
（このような、事前学習したモデルに後から畳み込み層と結合層を追加するやり方はパフォーマンスを上げてくれる、らしい。）
（詳細は「S. Ren, K. He, R. B. Girshick, X. Zhang, and J. Sun. Object　detection networks on convolutional feature maps. CoRR,abs/1504.06066, 2015. 3, 7」を参照）
画像分類は、「inputのサイズが大きい方がよろしい」ものらしいので、事前学習では224*224でやっていたが、448*448に修正した。


##### optimizer
φ(x) = x, if x > 0
            0.1x, otherwise

##### cost関数
二乗和誤差を使ったらしいが、論文中でもこれは「理想ではない」と言っている。
grid数を大きくすると、ほとんどのgridで「物体がない」状態になるが、その際に勾配消失が起きてしまい学習をやたらと早く収束させてしまう。
これに対応するために、下記の２つのパラメータを導入した。
物体がない場合はあまり学習に反映させない、物体があった場合の分類ミス等は大きく反映させるイメージ。
（この辺りはふんわり概要だけしか把握できていない。）


```math

λ_{coord}: 物体がある場合に、コスト関数を大きくする。
λ_{noobj}:物体がない場合にコストを下げる。

```


また、二乗和誤差だとバウンディングボックス（BB）が大きくても小さくても誤差は同じものとして計算する。
本アルゴリズムとしては、大きなBBには小さなBBよりも小さな誤差としてほしいので、BBのw,hはそのままの値ではなく平方根を利用した。


##### トレーニング中に最適化した関数について

長すぎるのでここでは数式は書かないが、概要だけ記述する。
コスト関数は「物体がgridセル中にある場合」は「誤分類の時のみ」コストを大きくする。
ちょっとまだ理解しきれていないので、後で勉強すること


##### 学習のあれこれ
エポック数:135

学習係数
最初は小さく設定する。そうでないと勾配消失が起きた。
最初の７５エポックは10^-2,次の３０は10^-3,最後の30は10^-4とした。

バッチサイズ
64

モーメンタム
0.9

decay
0.0005

dropout
最初の結合層のところで、半分のノードをドロップアウトする。

データ拡張
元データの２０%くらい作成



### 2-4.YOLOの制限
１つのgridで予測するBB数。  
また、各BBにおける予測クラスは１つのみになってしまう。

小さな物体（ex.鳥の大群）の識別が苦手。
なぜなら、誤差を「BBの大小に関わらず」同等のものとして扱ってしまうため。







<img src="./work/YOLO/network.png">

# 3.既存手法との比較

### 3-1.DPM(Deformable parts models)
YOLOでは、１つのネットワークで完結する点が異なる。
１つのネットワークで全てを完結させることで、最終的なパフォーマンス、精度の向上に繋がる。

### 3-2.R-CNN
R-CNNやその派生系も、特徴量抽出やBB検出、物体分類等で異なるモジュールを使うのでチューニングが大変かつ、処理が遅い。  
また、YOLOはその制約により、一つのgridに識別するBBの数を制限しているので、「同じ物体を重複して出力する」というR-CNNのデメリットを和らげている。  
共通点としては、各gridからBBを予測する際や各BBの信頼度計算時にCNNで特徴量を抽出しているところ。  

Fast R-CNNでも、結局処理が遅い。

### 3-3.Deep MultiBox
所詮物体検出をおこなう上でのパーツの１つに過ぎない。  
YOLOは物体検出において「完璧な」アルゴリズムであるとのこと。  
BBを推測する際にCNNで特徴量を抽出するところは共通点ではある。

### 3-4.OverFeat
物体検出と分類を同時にできるものではない。

### 3-5.MultiGrasp
全体的な思想は似ている。ただ、MultiGraspは処理できる内容が単純すぎる。  
（物体のサイズや他クラス分類ができない等）  
その点、YOLOは１つの画像で複数の物体＆複数のクラスへの分類が可能である。


# 4.検証
YOLOとR-CNNの派生系の中で一般的な「Fast R-CNN」の処理結果を比較する。  

### 4-1.リアルタイムで処理できるシステムとの比較
ほとんどの物体検出の研究は、「標準的な手法をより早く実行する」、という方針で進められている。  
その中で本当の意味で「リアルタイムで」処理できると思われるのは[Sadeghi et al」が提案した手法](M. A. Sadeghi and D. Forsyth. 30hz object detection with dpm v5. In Computer Vision–ECCV 2014, pages 65–79.Springer, 2014. 5, 6)で、この手法だと30fpsというパフォーマンスが証明されている。  
なので、YOLOと上記手法を比較した。他の手法は処理速度と精度の観点から比較するレベルに至っていないためである。  
「Fast YOLO」が一番処理が早いものの、精度(mAP)が52.7%であった。（既存のリアルタイム検出手法の２倍以上の精度）  
一方、YOLOはリアルタイムで処理できるパフォーマンスを維持しつつもmAPを63.4％にまで押し上げた。  

VGG-16を使ったYOLOも試したが、精度は向上したもののリアルタイム処理ができるほどのパフォーマンスが得られなかった。  
「Fastest DPM」は精度がそこまで落ちないものの、処理速度がまだリアルタイム処理できるほどのものではない。  
「Fast R-CNN」はmAPは高いが、リアルタイム処理とは程遠い。  
「Faster R-CNN」も同様に、まだリアルタイム処理とは程遠い。


##### リアルタイム処理ができると思われる手法

| 手法 | トレーニングデータ | mAP | FPS |
|---|---|---|
|100Hz DPM | 2007 | 16.0 | 100 |
|30Hz DPM | 2007 | 26.1 | 30 |
|Fast YOLO | 2007+2012 | 52.7 | 155 |
|YOLO | 2007+2012 | 63.4 | 45 |


##### リアルタイム処理はできなさそうな手法
| 手法 | トレーニングデータ | mAP | FPS |
|---|---|---|
|Fastest DPM | 2007 | 30.4 | 15 |
|R-CNN Minus R | 2007 | 53.5 | 6 |
|Fast R-CNN | 2007+2012 | 70.0 | 0.5 |
|Faster R-CNN VGG-16 | 2007+2012 | 73.2 | 7 |
|Faster R-CNN ZF | 2007+2012 | 62.1 | 18 |
|YOLO VGG-16 | 2007+2012 | 66.4 | 21 |


### 4-2.VOC 2007 Error Analysis
誤差の内容を簡単に調査しました。
評価指標は[ Hoiem et al.](D. Hoiem, Y. Chodpathumwan, and Q. Dai. Diagnosing error in object detectors. In Computer Vision–ECCV 2012, pages
340–353. Springer, 2012. 6)を参考にした。  

YOLOは「背景を物体と認識しない」点について高いパフォーマンスを発揮していた。  
「Fast R-CNN」はBBを作る点のパフォーマンスが高い。  


### 4-3.Fast R-CNNとYOLOを組み合わせてみた  

YOLOは「背景を物体と認識しない」点について精度が高いので、そのメリットを活かす形で「Fast R-CNN」と組み合わせたところ、「Fast R-CNN」の精度（mAP）が向上した。  
(ちょっとだけ)  
具体的には、「Fast R-CNN」が出力したBBをYOLOもBBとして出力しているか、というチェックを加えた。

結局２つのモデルを処理させることになるので処理時間に問題があるのだが、YOLOの処理時間は短いため「既に、Fast R-CNNを使っていた場合」、この組み合わせを利用することで精度向上を図ることができる。  


### 4-4.VOC 2012 Results
VOC 2012のデータセットでも検証した。  
YOLOのmAPは57.9%で、他の手法よりも低かった。  
クラスごとに見てみると、「ボトル」や「羊」等の小さい物質の検出にミスが多いようだった。  
逆に「猫」や「電車」等のクラスの分類はよくできていた。  
「Fast R-CNN」と「YOLO」を組み合わせたモデルは、トップクラスのパフォーマンスを発揮した。  


### 4-5.美術作品の作者分類
現実的には、このような物体検出モデルは利用ケースが多岐にわたるため適用が難しい。  
そこで、ピカソと、「人物画」のデータを分類できるか試してみた。  






# 備考

##### IoU(Intersection over Union)
[このブログ](https://meideru.com/archives/3538)の解説がとてもわかりやすくて良い。  


##### Non-Maximum Suppression

論文中でもさらっと出てきているが、これはなんなのか。  
[このブログ](https://meideru.com/archives/3538)の解説がとてもわかりやすくて良い。  
同じクラスと予測したBBが重複している際、一つの領域として調整するアルゴリズムのようです。

IoU値が閾値よりも大きい場合は、その領域を抑制（suppression）する、というもの。




